In [ ]:
# pip install plotly


In [ ]:
import plotly.express as px

# 创建 treemap 图表,定义好父子关系
fig = px.treemap(
    names=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve"]
)

# 更新节点颜色
fig.update_traces(root_color="lightgrey")

# 更新布局设置
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

# 显示图表
fig.show()
# 这段代码使用 Plotly Express 创建了一个 treemap 图表，其中 names 列表指定了节点的名称，parents 列表指定了节点的父节点。
# 通过 update_traces 方法可以设置根节点的颜色为 "lightgrey"。
# update_layout 方法用于更新图表的布局，其中 margin 参数设置了图表的上、左、右、下的边距。
# 最后，使用 fig.show() 显示图表。



In [ ]:
import plotly.express as px

# 使用 tips 数据集作为示例数据
df = px.data.tips()

# 创建 treemap 图表，指定路径和值，按照 day、time、sex 的关系进行表达
fig = px.treemap(df, path=[px.Constant("all"), 'day', 'time', 'sex'], values='total_bill')

# 创建每个层级对应的字体大小列表
sizes = [30, 20, 10]

# 通过遍历每个层级，手动设置字体大小
for i, size in enumerate(sizes):
    fig.update_traces(
        textfont=dict(size=size),
        selector=dict(type="treemap", level=i)
    )

# 更新节点颜色
fig.update_traces(root_color="lightgrey")

# 更新布局设置
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    # update_layout 方法用于更新 Plotly 图表的整体布局，
    # 其中 margin 参数用于设置图表的上、左、右和下边距的大小。
    # 在这里，dict(t=50, l=25, r=25, b=25) 
    # 表示设置上边距（top）为 50，左边距（left）为 25，右边距（right）为 25，下边距（bottom）为 25。
# 显示图表


fig.show()



In [ ]:
df

In [ ]:
import plotly.express as px
import numpy as np

# 从 gapminder 数据集中选择 2007 年的数据
df = px.data.gapminder().query("year == 2007")

# 创建 treemap 图表，指定路径、值、颜色和悬停数据
fig = px.treemap(df, path=[px.Constant("world"), 'continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))

# 更新布局设置
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))


# 显示图表
fig.show()
'''values 参数指定了 treemap 的值字段为 'pop'（人口数量）。
color 参数设置了根据 'lifeExp'（人均寿命）的值给区域着色。
hover_data 参数指定了鼠标悬停时显示的附加数据，这里选择了 'iso_alpha'（国家的 ISO Alpha-2 代码）。
color_continuous_scale 参数设置了颜色的渐变色表，这里使用的是 'RdBu'。
color_continuous_midpoint 参数设置了颜色的中点，使用了人均寿命的加权平均值。
'''


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/sales_success.csv')
print(df.head())

levels = ['salesperson', 'county', 'region'] # levels used for the hierarchical chart
color_columns = ['sales', 'calls']
value_column = 'calls'

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)
average_score = df['sales'].sum() / df['calls'].sum()

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    name=''
    ), 1, 1)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    maxdepth=2
    ), 1, 2)

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 从指定 URL 读取数据集
df = pd.read_csv("coffee.csv")

# 创建图表对象
fig = go.Figure()

# 添加 Treemap 图表
fig.add_trace(go.Treemap(
    ids=df.ids,
    labels=df.labels,
    parents=df.parents,
    maxdepth=2, # 最多同时出现3级
    root_color="lightgrey"
))

# 更新布局设置
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

# 显示图表
fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 从指定 URL 读取数据集
df = pd.read_csv("人工智能的介绍和应用大纲.csv")

# 创建图表对象
fig = go.Figure()

# 添加 Treemap 图表
fig.add_trace(go.Treemap(
    ids=df.ids,
    labels=df.labels,
    parents=df.parents,
    maxdepth=2, # 最多同时出现3级
    root_color="lightgrey"
))

# 更新布局设置
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

# 显示图表
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 假设我们有这样一个数据集
df = pd.DataFrame({
    'ids': ['A', 'B', 'C', 'D', 'E', 'F'],
    'labels': ['A', 'B', 'C', 'D', 'E', 'F'],
    'parents': ['', 'A', 'A', 'B', 'B', 'C'],
    'extra_info': ['info A', 'info B', 'info C', 'info D', 'info E', 'info F']
})

# 创建初始的treemap
fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'],  # 初始时，这里的信息会被显示出来
))

# 更新layout以增加交互性
fig.update_layout(
    hovermode='closest',
    treemapcolorway=["#0f7216", "#b2790c", "#ffe9a3"],
    autosize=True,
)

# 注册点击事件
def update_point(trace, points, selector):
    # 当点击事件发生时，改变text的信息
    c = list(trace.text)
    for i in points.point_inds:
        c[i] = 'new info'  # 这里是新的信息
    trace.text = c

# 在图形中的每个trace上绑定这个事件
for trace in fig.data:
    trace.on_click(update_point)

fig.show()


In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd

app = JupyterDash(__name__)

# 假设我们有这样一个数据集
df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 'Information B', 'Information B1', 'Information B2', 'Information C', 'Information C1']
})

# 创建初始的treemap
fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'],  # 初始时，这里的信息会被显示出来
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'})
])

@app.callback(
    Output('my-graph', 'figure'),
    Input('my-graph', 'clickData'),
    State('my-graph', 'figure'))
def update_figure(clickData, fig):
    # 当点击事件发生时，改变显示的信息
    if clickData:
        point_number = clickData['points'][0]['pointNumber']
        fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']
    return fig

app.run_server(mode='inline')


In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json

app = JupyterDash(__name__)

# 假设我们有这样一个数据集
df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
       'click_info': ['Information A', 
                   'Information B', 
                   'Information B1', 
                   'Information B2', 
                   'Information C', 
                   'Information C1']
})

# 创建初始的treemap
fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'].tolist()  # 初始时，这里的信息会被显示出来
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'}),
    html.Div(id='text-storage', style={'display': 'none'})
])

@app.callback(
    Output('my-graph', 'figure'),
    Output('text-storage', 'children'),
    Input('my-graph', 'clickData'),
    State('my-graph', 'figure'),
    State('text-storage', 'children'))
def update_figure(clickData, fig, stored_text):
    # 当点击事件发生时，改变显示的信息
    if clickData:
        point_number = clickData['points'][0]['pointNumber']
        # 如果这是第一次点击，保存原始的文本
        if stored_text is None:
            stored_text = json.dumps(fig['data'][0]['text'])
        # 更新点击的节点的文本
        fig['data'][0]['text'] = json.loads(stored_text)  # 首先恢复所有的节点的文本
        fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']  # 然后更新点击的节点的文本
    else:
        # 如果点击了空白区域，恢复原始的文本
        if stored_text is not None:
            fig['data'][0]['text'] = json.loads(stored_text)
    return fig, stored_text

app.run_server(mode='inline')
# 切换其他节点时会恢复原始的文本

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json

app = JupyterDash(__name__)

# 假设我们有这样一个数据集
df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 
                   'Information B', 
                   'Information B1', 
                   'Information B2', 
                   'Information C', 
                   'Information C1']
})

# 创建初始的treemap
fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],
    text=df['extra_info'].tolist()  # 初始时，这里的信息会被显示出来
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'}),
    html.Div(id='text-storage', style={'display': 'none'}),
    html.Div(id='node-clicked', style={'display': 'none'})
])

@app.callback(
    Output('my-graph', 'figure'),
    Output('text-storage', 'children'),
    Output('node-clicked', 'children'),
    Input('my-graph', 'clickData'),
    State('my-graph', 'figure'),
    State('text-storage', 'children'),
    State('node-clicked', 'children'))
def update_figure(clickData, fig, stored_text, clicked_node):
    # 当点击事件发生时，改变显示的信息
    if clickData:
        point_number = clickData['points'][0]['pointNumber']
        # 判断是否再次点击了同一个节点
        if clicked_node is None or clicked_node != point_number:
            # 更新点击的节点的文本
            fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']  # 更新为 "Information"
            clicked_node = point_number
        else:
            # 如果再次点击了同一个节点，恢复原始的文本
            fig['data'][0]['text'][point_number] = df.loc[point_number, 'extra_info']  # 更新为 "info"
            clicked_node = None
    else:
        # 如果点击了空白区域，恢复原始的文本
        if stored_text is not None:
            fig['data'][0]['text'] = json.loads(stored_text)
        clicked_node = None

    return fig, json.dumps(fig['data'][0]['text']), clicked_node

app.run_server(mode='inline')
# 再次点击会恢复原始的文本

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json

app = JupyterDash(__name__)

df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 'Information B', 'Information B1', 'Information B2', 'Information C', 'Information C1']
})

fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'].tolist()
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'}),
    html.Div(id='text-storage', style={'display': 'none'}),
    html.Div(id='node-clicked', style={'display': 'none'})
])

@app.callback(
    Output('my-graph', 'figure'),
    Output('text-storage', 'children'),
    Output('node-clicked', 'children'),
    Input('my-graph', 'clickData'),
    State('my-graph', 'figure'),
    State('text-storage', 'children'),
    State('node-clicked', 'children'))
def update_figure(clickData, fig, stored_text, clicked_node):
    if clickData:
        point_number = clickData['points'][0]['pointNumber']
        if stored_text is None:
            stored_text = json.dumps(fig['data'][0]['text'])
        if clicked_node is None or clicked_node != point_number:
            fig['data'][0]['text'] = json.loads(stored_text)
            fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']
            clicked_node = point_number
        else:
            fig['data'][0]['text'] = json.loads(stored_text)
            clicked_node = None
    else:
        if stored_text is not None:
            fig['data'][0]['text'] = json.loads(stored_text)
        clicked_node = None

    return fig, stored_text, clicked_node

app.run_server(mode='inline')
# 功能合并版本

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import dash

app = JupyterDash(__name__)

df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 'Information B', 'Information B1', 'Information B2', 'Information C', 'Information C1']
})

fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'].tolist()
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'}),
    html.Div(id='text-storage', style={'display': 'none'}),
    html.Div(id='node-clicked', style={'display': 'none'}),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds
        max_intervals=0
    )
])

@app.callback(
    Output('my-graph', 'figure'),
    Output('text-storage', 'children'),
    Output('node-clicked', 'children'),
    Output('interval-component', 'max_intervals'),
    Input('my-graph', 'clickData'),
    Input('interval-component', 'n_intervals'),
    State('my-graph', 'figure'),
    State('text-storage', 'children'),
    State('node-clicked', 'children'))
def update_figure(clickData, n_intervals, fig, stored_text, clicked_node):
    ctx = dash.callback_context
    if ctx.triggered[0]['prop_id'] == 'my-graph.clickData':
        point_number = clickData['points'][0]['pointNumber']
        if stored_text is None:
            stored_text = json.dumps(fig['data'][0]['text'])
        fig['data'][0]['text'] = json.loads(stored_text)
        if clicked_node is None or clicked_node != point_number:
            fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']
            clicked_node = point_number
            max_intervals = 1
        else:
            clicked_node = None
            max_intervals = 0
    elif ctx.triggered[0]['prop_id'] == 'interval-component.n_intervals':
        fig['data'][0]['text'] = json.loads(stored_text)
        clicked_node = None
        max_intervals = 0
    else:
        if stored_text is not None:
            fig['data'][0]['text'] = json.loads(stored_text)
        clicked_node = None
        max_intervals = 0

    return fig, stored_text, clicked_node, max_intervals

app.run_server(mode='inline')


In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import dash

app = JupyterDash(__name__)

df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 'Information B', 'Information B1', 'Information B2', 'Information C', 'Information C1']
})

fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'].tolist()
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig,
        config={'doubleClick': 'reset'}  # Reset clickData on double click
    ),
    html.Div(id='click-data', style={'display': 'none'}),
    html.Button(id='reset-button', style={'display': 'none'}),
    dcc.Interval(
        id='interval-component',
        interval=200,  # 200 ms
        n_intervals=0,
        max_intervals=1  # Only run once after a delay
    )
])

@app.callback(
    Output('click-data', 'children'),
    Output('interval-component', 'max_intervals'),
    Input('my-graph', 'clickData'),
    Input('interval-component', 'n_intervals'),
    prevent_initial_call=True  # Prevent this callback from being triggered on app startup
)
def store_click_data(clickData, n_intervals):
    ctx = dash.callback_context
    if ctx.triggered[0]['prop_id'] == 'my-graph.clickData':
        return json.dumps(clickData), 0  # Reset the interval
    elif ctx.triggered[0]['prop_id'] == 'interval-component.n_intervals':
        return dash.no_update, 1  # Stop the interval
    else:
        return dash.no_update, dash.no_update

@app.callback(
    Output('my-graph', 'figure'),
    Output('my-graph', 'clickData'),
    Input('click-data', 'children'),
    State('my-graph', 'figure'))
def update_figure(stored_clickData, fig):
    if stored_clickData:
        clickData = json.loads(stored_clickData)
        point_number = clickData['points'][0]['pointNumber']
        text = df['extra_info'].tolist()
        text[point_number] = df.loc[point_number, 'click_info']
        fig['data'][0]['text'] = text
    return fig, None  # Reset clickData

app.run_server(mode='inline')
